In [68]:
import torch_geometric.transforms
from torch_geometric.nn import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import numpy as np
import pickle
from torch_geometric.data import DataLoader
import linecache
import random

In [69]:
pickle_in2 = open("defect_matrices/dict.eq_distr","rb")
eq_distr = []
while 1:
    try:
        eq_distr.append(pickle.load(pickle_in2))
    except EOFError:
        break

print(len(eq_distr))

100


In [70]:
node_feat_load = np.genfromtxt('defect_matrices/node_features.txt')
edge_idx_load = np.genfromtxt('defect_matrices/edgelist.txt',usecols=(0,1))
edge_feat_load = np.genfromtxt('defect_matrices/edgelist.txt',dtype = float,usecols=3,comments = '}')
graph_info = np.genfromtxt('defect_matrices/graph_info.txt')

In [78]:
graphs = []

holder=0
edge_holder =0
for ix in range(len(graph_info)):
    num_nodes = int(graph_info[ix])
    edge_amount = int(num_nodes*(num_nodes-1))
    
    node_feat = node_feat_load[holder:holder+num_nodes]
    edge_idx = edge_idx_load[edge_holder:edge_amount+edge_holder]
    edge_feat = edge_feat_load[edge_holder:edge_amount+edge_holder]
    
    node_features = torch.from_numpy(node_feat)
    edge_features = torch.from_numpy(edge_feat)
    
    edge_index = torch.from_numpy(np.transpose(edge_idx)).type(torch.LongTensor)
    #edge_index = torch_geometric.utils.to_undirected(edge_index)
    
    #print(node_features)
    #print(edge_index)
    #print(edge_features.shape)
    
    eq_class = np.array(np.around(eq_distr[ix]*0.01))
    eq_class = torch.from_numpy(eq_class)

    graph = Data(node_features,edge_index,edge_features,y=eq_class)
    #print(graph)
    graphs.append(graph)
    
    holder = holder + num_nodes
    edge_holder = edge_holder + edge_amount


    

torch.save(graphs,'defect_matrices/torch_test.pt')  


tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]])


In [5]:
    eq_class = np.array(eq_distr[1])
    eq_class = torch.from_numpy(eq_class)

In [7]:
print(np.around(eq_class*0.01))

tensor([1., 0., 0., 0.], dtype=torch.float64)


In [79]:
dataset = torch.load('defect_matrices/torch_test.pt')


In [80]:

#print()
#print(f'Dataset: {dataset}:')
#print('====================')
print(f'Number of graphs: {len(dataset)}')
print()
#print(f'Number of features: {dataset.num_features}')
#print(f'Number of classes: {dataset.num_classes}')
print(f'First graph attributes:')
data = dataset[0]  # Get the first graph object.
#print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')


Number of graphs: 98

First graph attributes:
Data(edge_attr=[72], edge_index=[2, 72], x=[9, 2], y=[4])
Number of nodes: 9
Number of edges: 72
Average node degree: 8.00


In [196]:
torch.manual_seed(1234)
#dataset = dataset.shuffle()
random.shuffle(dataset)

train_dataset = dataset[:8]
test_dataset = dataset[8:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')



Number of training graphs: 8
Number of test graphs: 2


In [198]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 4
Batch(batch=[41], edge_attr=[216], edge_index=[2, 216], x=[41, 2], y=[16])

Step 2:
Number of graphs in the current batch: 4
Batch(batch=[27], edge_attr=[81], edge_index=[2, 81], x=[27, 2], y=[16])



In [240]:
print(batch)

NameError: name 'batch' is not defined

In [300]:
from torch.nn import Linear
from torch.nn import Softmax

import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        print(dataset.num_features)
        
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        print(x.shape)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)
    


GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [263]:
output = torch.randn(5,10)
target = torch.randint(0,4,(5,))

print(output.shape)
print(target)


torch.Size([5, 10])
tensor([3, 1, 0, 1, 0])


In [296]:
print(out.shape)
print(data.y)

torch.Size([17, 2])
tensor([1])


In [301]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        print(out.shape)
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 2):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

7
torch.Size([64, 64])
torch.Size([64, 2])
7
torch.Size([64, 64])
torch.Size([64, 2])
7
torch.Size([22, 64])
torch.Size([22, 2])
7
torch.Size([64, 64])
7
torch.Size([64, 64])
7
torch.Size([22, 64])
7
torch.Size([38, 64])
Epoch: 001, Train Acc: 0.6467, Test Acc: 0.7368


In [302]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2

Data(edge_attr=[38, 4], edge_index=[2, 38], x=[17, 7], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [291]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Number of training graphs: 150
Number of test graphs: 38


In [304]:
print()

NameError: name 'y' is not defined

In [289]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 4
Batch(batch=[70], edge_attr=[152, 4], edge_index=[2, 152], x=[70, 7], y=[4])

Step 2:
Number of graphs in the current batch: 4
Batch(batch=[83], edge_attr=[190, 4], edge_index=[2, 190], x=[83, 7], y=[4])

Step 3:
Number of graphs in the current batch: 4
Batch(batch=[88], edge_attr=[194, 4], edge_index=[2, 194], x=[88, 7], y=[4])

Step 4:
Number of graphs in the current batch: 4
Batch(batch=[59], edge_attr=[124, 4], edge_index=[2, 124], x=[59, 7], y=[4])

Step 5:
Number of graphs in the current batch: 4
Batch(batch=[89], edge_attr=[196, 4], edge_index=[2, 196], x=[89, 7], y=[4])

Step 6:
Number of graphs in the current batch: 4
Batch(batch=[66], edge_attr=[146, 4], edge_index=[2, 146], x=[66, 7], y=[4])

Step 7:
Number of graphs in the current batch: 4
Batch(batch=[88], edge_attr=[202, 4], edge_index=[2, 202], x=[88, 7], y=[4])

Step 8:
Number of graphs in the current batch: 4
Batch(batch=[58], edge_attr=[124, 4], edge_index=[2, 124], x=[

In [287]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [288]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()

ValueError: Expected input batch_size (17) to match target batch_size (1).